# Clustering
## Input
We will cluster the sequences using MMseqs2. First we need to combine the sequences we downloaded into one file.

In [4]:
import os
from Bio import SeqIO
from collections import defaultdict

In [5]:
seq_file = "all_proteins_combined.fasta"
data_dir = "../data"
cluster_file = "mmseqs_output_cluster.tsv"
path_raw_gene_families = "../raw_gene_families"
path_clean_families = "../filtered_gene_families"

In [6]:
#combine all fasta files for mmseqs


print(f"Creating {seq_file}...")

with open(seq_file, "w") as outfile:
    count = 0
    for filename in os.listdir(data_dir):
        if filename.endswith(".faa"):
            #use filename as species label (like Bifidobacterium_longum)
            species_tag = filename.replace(".faa", "")
            
            with open(os.path.join(data_dir, filename), "r") as infile:
                for line in infile:
                    if line.startswith(">"):
                        
                        original_id = line.split()[0].lstrip(">")
                        outfile.write(f">{species_tag}|{original_id}\n")
                    else:
                        outfile.write(line)
            count += 1

print(f"Merged {count} genomes into {seq_file}")

Creating all_proteins_combined.fasta...
Merged 55 genomes into all_proteins_combined.fasta


## Running clustering script
Now we run MMseqs2 on all_proteins_combined.fasta using *run_mmseqs.py*. I decided to do this via a separate script so that I can run it on the Entropy compute cluster and make it faster.

On the cluster I simply open a bash session

*srun --pty --cpus-per-task=4 --time=100 bash*

and run the script

*time python3 run_mmseqs.py*

which utilzed only took 1min 11s, thanks to utilizing 40 CPU cores.



## Output
We obtain 3 files:
* mmseqs_output_cluster.tsv - this file says which sequence belongs to which cluster. Contains two columns: cluster name (a cluster is named after one of its representatives) and name of the sequence belonging to it. 
* mmseqs_output_rep_seq.fasta - those representative sequences clusters are named after 
* mmseqs_output_all_seqs.fasta - all the sequences reorganized in accordance with the clustering.


### Gene families

In [7]:

#load clusters
seq_dict = SeqIO.to_dict(SeqIO.parse(seq_file, "fasta"))
clusters = defaultdict(list)
with open(cluster_file) as f:
    for line in f:
        rep, member = line.strip().split('\t')
        clusters[rep].append(member)

n_clusters = len(clusters)
print(f"{n_clusters} clusters in total")


28757 clusters in total


In [9]:
#write them to indivitual files

for i, (rep, members) in enumerate(clusters.items()):
    if not i%1000:
        print(f"{i}/{n_clusters}")

    records = [seq_dict[m] for m in members if m in seq_dict]
    
    #replace special characters for valid file name
    safe_name = rep.replace("|", "_").replace("/", "_")
    output_path = os.path.join(path_raw_gene_families, f"cluster_{safe_name}.fasta")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    SeqIO.write(records, output_path, "fasta")

0/28757
1000/28757
2000/28757
3000/28757
4000/28757
5000/28757
6000/28757
7000/28757
8000/28757
9000/28757
10000/28757
11000/28757
12000/28757
13000/28757
14000/28757
15000/28757
16000/28757
17000/28757
18000/28757
19000/28757
20000/28757
21000/28757
22000/28757
23000/28757
24000/28757
25000/28757
26000/28757
27000/28757
28000/28757


In [12]:
#filtering paralogs and renaming the sequences to contain only names of the species
os.makedirs(path_clean_families, exist_ok=True)
os.makedirs(path_raw_gene_families, exist_ok=True)

n = len(clusters)

for i, (rep, members) in enumerate(clusters.items()):
    if not i % 1000:
        print(f"{i}/{n}")

    species_best = {}
    all_records = []

    for member in members:
        if member not in seq_dict: continue
        
        # parse id: Species|ProteinID
        parts = member.split('|')
        species = parts[0]
        pid = parts[1] if len(parts) > 1 else "unk"
        
        record = seq_dict[member]

        # 1. filter best (longest)
        if species not in species_best or len(record.seq) > len(species_best[species].seq):
            species_best[species] = record

        # 2. rename paralogs (unique headers required)
        raw_rec = record[:] 
        raw_rec.id = f"{species}_{pid}" 
        raw_rec.description = ""
        all_records.append(raw_rec)

    # save clean (1-to-1)
    clean_records = []
    for species, record in species_best.items():
        new_rec = record[:] 
        new_rec.id = species
        new_rec.description = "" 
        clean_records.append(new_rec)

    safe_name = rep.replace("|", "_").replace("/", "_")
    
    if clean_records:
        SeqIO.write(clean_records, os.path.join(path_clean_families, f"cluster_{safe_name}.fasta"), "fasta")

    if all_records:
        SeqIO.write(all_records, os.path.join(path_raw_gene_families, f"cluster_{safe_name}.fasta"), "fasta")

0/28757
1000/28757
2000/28757
3000/28757
4000/28757
5000/28757
6000/28757
7000/28757
8000/28757
9000/28757
10000/28757
11000/28757
12000/28757
13000/28757
14000/28757
15000/28757
16000/28757
17000/28757
18000/28757
19000/28757
20000/28757
21000/28757
22000/28757
23000/28757
24000/28757
25000/28757
26000/28757
27000/28757
28000/28757


# Alignments
Now we will align the sequences using MAFFT. Both for the gene families with paralogs removed (*filtrered_gene_families*) and ones with paralogs present (*raw_gena_families*), in case we wanna use them later. Took 27min 16s.


# Gene trees
Computing gene trees with FastTree took 5min 39s. Computations were run togther for filtered_gene_families (no paralogs) and raw_gene_families (with paralogs), because if there's less then 4 trees in the filtered version of a gene family, we remove the unfiltered version too, since it contains less than 4 unique species.